# pulse2percept: Edema Example




In [ ]:
import skimage.io as skio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pulse2percept as p2p

In [ ]:
# Load an Argus II array
argus = p2p.implants.ArgusII( h=0, rot=np.deg2rad(-35), eye='RE')

In [ ]:
# Start the simulation framework
sim = p2p.Simulation(argus, engine='joblib', n_jobs=1)

In [ ]:
#Set up an edema_map
s_sample = 100
xhi = 1000
xlo = -1000
xrange = (xlo, xhi)
yhi = 1000
ylo = -1000
yrange = (ylo, yhi)
sampling=s_sample
num_x = int((xhi - xlo) / sampling + 1)
num_y = int((yhi - ylo) / sampling + 1)

gridx, gridy = np.meshgrid(np.linspace(xlo, xhi, num_x),
                                             np.linspace(ylo, yhi, num_y),
                                             indexing='xy')

edema_map = np.zeros((gridx.shape[0], gridx.shape[1]), dtype=float)
edema_map[12,:] = 1
edema_map.shape


In [ ]:
# Set parameters of the optic fiber layer (OFL)
# In previous versions of the model, this used to be called the `Retina`
# object, which created a spatial grid and generated the axtron streak map.

# Set the spatial sampling step (microns) of the retinal grid
sim.set_optic_fiber_layer(sampling=s_sample, n_axons=501, n_rho=801, rho_range=(4, 45),
                          sensitivity_rule='decay', decay_const=5.0,
                          contribution_rule='max', edema_map = edema_map)

In [ ]:
# Set parameters of the ganglion cell layer (GCL)
# In previous versions of the model, this used to be called `TemporalModel`.

# Set the temporal sampling step (seconds)
t_sample = 0.01 / 1000
sim.set_ganglion_cell_layer('Nanduri2012', tsample=t_sample)

### 1.4 Specifying the input stimulus

In [ ]:
# Send a pulse train to two specific electrodes, set all others to zero
stim = {
    'D4': p2p.stimuli.PulseTrain(t_sample, freq=50, amp=20, dur=0.5), 
}

We can visualize the specified electrode array and its location on the retina with respect to the optic disc:

In [ ]:
p2p.viz.plot_fundus(argus, stim, upside_down=False);

# 2. Running the Simulation

Simulations are run by passing an input stimulus `stim` to `p2p.pulse2percept`. Output is a `p2p.utils.TimeSeries` objects that contains the brightness changes over time for each pixel.

In [ ]:
# Run a simulation
# - tol: ignore pixels whose efficient current is smaller than 10% of the max
# - layers: simulate ganglion cell layer (GCL) and optic fiber layer (OFL),
#   but ignore inner nuclear layer (INL) for now
percept = sim.pulse2percept(stim, tol=0.1, layers=['GCL', 'OFL'])

In [ ]:
debug


In [ ]:
frame = p2p.get_brightest_frame(percept)
plt.figure(figsize=(8, 5))
plt.imshow(frame.data, cmap='gray')
plt.colorbar()

In [ ]:
cs = np.zeros((gridx.shape[0], gridx.shape[1],
                       2, len(argus.electrodes)), dtype=float)
for i, e in enumerate(argus.electrodes):
            cs[..., 0, i] = e.current_spread(gridx, gridy,
                                             layer='INL', alpha=14000)
            cs[..., 1, i] = e.current_spread(gridx, gridy,
                                             layer='OFL', alpha=14000)
        
skio.imshow(cs[:,:,1,10])

In [ ]:
edemas = p2p.retina.edema_contribution(cs[:,:,1,10],edema_map)
edemas.shape
skio.imshow(edemas)

In [ ]:
p2p.implants.Electrode.current_spread